#### Find kernel with highest accuracy within Gaussian Process Regressor
#### Cross-validated randomised search of hyperparameters
#### For 20 random test grid cells, each with 500 iterations

In [12]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split,
                                     cross_val_score,
                                     RandomizedSearchCV,
                                     LeaveOneOut)
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import (DotProduct,
                                              ConstantKernel,
                                              WhiteKernel,
                                              Matern,
                                              ExpSineSquared,
                                              RationalQuadratic,
                                              RBF)

In [3]:
with open('/nfs/see-fs-02_users/earlacoa/emulator/dfs_gridcell_sample.pickle', 'rb') as ds:
    df_gridcells = pickle.load(ds)
    
df_gridcells.head()

,lat,lon,RES,IND,TRA,AGR,POW,PM2_5_DRY
0,30.5,114.25,0.23229,0.20507,0.299040,0.97374,0.55184,82.050903
1,30.5,114.25,1.12120,1.10980,0.870100,1.22220,1.25200,260.182960
2,30.5,114.25,0.79843,1.29460,0.090709,0.75401,0.19420,209.802636
3,30.5,114.25,0.34445,1.42240,0.751120,1.24090,1.21130,164.002287
4,30.5,114.25,0.62562,1.35890,1.058300,0.04688,0.64196,196.083809


In [9]:
kernel_list = [Matern(nu=i, length_scale=j) for i in [0.5, 1.5, 2.5] for j in np.arange(0.1, 5, 0.1)] + \
              [ConstantKernel(constant_value=i) ** Matern(nu=j, length_scale=k) for i in np.arange(0.1, 5, 0.1) for j in [0.5, 1.5, 2.5] for k in np.arange(0.1, 5, 0.1)] + \
              [ConstantKernel(i) for i in np.arange(0.1, 5, 0.1)] + \
              [DotProduct(i) for i in np.arange(0.1, 5, 0.1)] + \
              [ConstantKernel(i) ** DotProduct(j) for i in np.arange(0.1, 5, 0.1) for j in np.arange(0.1, 5, 0.1)] + \
              [WhiteKernel(i) for i in np.arange(0.1, 5, 0.1)] + \
              [ExpSineSquared(i) for i in np.arange(0.1, 5, 0.1)] + \
              [RationalQuadratic(i) for i in np.arange(0.1, 5, 0.1)] + \
              [RBF(i) for i in np.arange(0.1, 5, 0.1)]

In [5]:
param_grid = {'kernel': kernel_list,
              'n_restarts_optimizer': np.arange(100, 200, 5),
              'normalize_y': [True, False]}

In [13]:
lats = df_gridcells[['lat', 'lon']].drop_duplicates()['lat'].values
lons = df_gridcells[['lat', 'lon']].drop_duplicates()['lon'].values

features = ['RES', 'IND', 'TRA', 'AGR', 'POW']
target = 'PM2_5_DRY'

for gridcell in df_gridcells[['lat', 'lon']].drop_duplicates().values:
    lat, lon = gridcell
    df_gridcell = df_gridcells.loc[df_gridcells.lat == lat].loc[df_gridcells.lon == lon]
    
    X = df_gridcell[features].values
    y = df_gridcell[target].values
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.33, random_state=123
    )
    
    loo = LeaveOneOut()
    #cv = loo.get_n_splits(X_train) # causes all NaNs at the moment, so using 5-fold for the time being
    cv = 5
    
    gp = GaussianProcessRegressor(random_state=123)
    emulator = RandomizedSearchCV(
        gp,         
        param_grid,
        cv=cv,
        n_jobs=-1,         
        n_iter=500,   
        random_state=123,                    
        verbose=2)
    emulator.fit(X_train, y_train)

    print(f"CV score: {emulator.best_score_:.4f}")
    print(f"test score: {emulator.score(X_test, y_test):.4f}")
    print(f"best estimator: {emulator.best_estimator_}")

Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 301 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 584 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 949 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 1394 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 1921 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 2500 out of 2500 | elapsed:  3.6min finished
/nfs/a68/earlacoa/miniconda2/envs/python3/lib/python3.6/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


CV score: -26.8291
test score: -0.9143
best estimator: GaussianProcessRegressor(alpha=1e-10, copy_X_train=True, kernel=0.316**2,
                         n_restarts_optimizer=120, normalize_y=False,
                         optimizer='fmin_l_bfgs_b', random_state=123)
Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 301 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 584 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 949 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 1394 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 1921 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 2500 out of 2500 | elapsed:  3.7min finished
/nfs/a68/earlacoa/miniconda2/envs/python3/lib/python3.6/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


CV score: -3.4044
test score: -0.8982
best estimator: GaussianProcessRegressor(alpha=1e-10, copy_X_train=True, kernel=1.9**2,
                         n_restarts_optimizer=100, normalize_y=False,
                         optimizer='fmin_l_bfgs_b', random_state=123)
Fitting 5 folds for each of 500 candidates, totalling 2500 fits


...

CV score: -1.9908
test score: -0.9350
best estimator: GaussianProcessRegressor(alpha=1e-10, copy_X_train=True, kernel=1.95**2,
                         n_restarts_optimizer=135, normalize_y=False,
                         optimizer='fmin_l_bfgs_b', random_state=123)
Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 301 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 584 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 949 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 1394 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 1921 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 2500 out of 2500 | elapsed:  3.8min finished
/nfs/a68/earlacoa/miniconda2/envs/python3/lib/python3.6/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


CV score: -723.2141
test score: -0.1420
best estimator: GaussianProcessRegressor(alpha=1e-10, copy_X_train=True, kernel=0.548**2,
                         n_restarts_optimizer=180, normalize_y=False,
                         optimizer='fmin_l_bfgs_b', random_state=123)
Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 301 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 584 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 949 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 1394 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 1921 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 2500 out of 2500 | elapsed:  4.0min finished
/nfs/a68/earlacoa/miniconda2/envs/python3/lib/python3.6/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


CV score: -24.3838
test score: -0.8828
best estimator: GaussianProcessRegressor(alpha=1e-10, copy_X_train=True, kernel=1.9**2,
                         n_restarts_optimizer=100, normalize_y=False,
                         optimizer='fmin_l_bfgs_b', random_state=123)
